### **To Do**
- try different data normalization (mean and std?)


### Imports and loading

In [1]:
# imports

import io
import os
import sys
from datetime import datetime
from time import time
import pickle
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import copy
import json
import matplotlib.pyplot as plt
import numpy as np
# from scipy.fft import fft
import pandas as pd
import seaborn as sns
from scipy import interpolate
from scipy.interpolate import interp1d
from scipy.spatial.distance import cdist
from scipy.stats import pearsonr

# Machine Learning
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

# torch
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import Sequential
from torch.nn import Sigmoid,ReLU
from torch.nn import Linear,Dropout
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor,Compose
from torch.optim import SparseAdam,Adam,Adagrad,SGD

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# prompt: Access a folder on google drive and import the data locally
from google.colab import drive
drive.mount('/content/drive/')
# helper files
sys.path.append('/content/drive/MyDrive/Final Project UAV/')
from UAV_project_preprocessing_and_visualization_helper_functions_full import *

Mounted at /content/drive/


In [2]:
cd /content/drive/MyDrive/Final Project UAV/

/content/drive/MyDrive/Final Project UAV


In [3]:
# file_path = "track_data/uav/VIS_uav_20230605-108_20230605-130248.json"
# file_path = "track_data/bird/VIS_bird_20230605-406_20230605-142825.json"
# file_path = "track_data/bird/VIS_bird_20231024-22251_20231024-083633.json"
# file_path = "track_data/airplane/VIS_airplane_20231023-6210_20231023-162937.json"
# file_path = "track_data/airplane/VIS_airplane_20231025-21925_20231025-074948.json"
# file_path = "track_data/uav/VIS_uav_20231027-10903_20231027-114017.json"
# file_path = "track_data/uav/VIS_uav_20231027-11132_20231027-115333.json"
# file_path = "track_data/uav/VIS_uav_20231027-10582_20231027-113033.json"
# file_path = "track_data/uav/VIS_uav_20231030-6113_20231030-140702.json"
# file_path = "track_data/uav/VIS_uav_20231027-10437_20231027-112435.json"
# file_path = "track_data/uav/VIS_uav_20230605-108_20230605-130248.json"
# file_path = "track_data/uav/VIS_uav_20230605-108_20230605-130248.json"
# plot_vs_time(file_path, 'vel')

### Segmentation

Assume interpolation of dt = 40 msec.\
For example: a sequence of 5 seconds is equivalent to 25*5 = 125

In [4]:
folder = 'track_data'

In [5]:
def segment_file_by_time(file_path, samples_config):
  sample_duration = samples_config['sample_duration']
  overlap_factor = samples_config['overlap_factor']
  tt, xx, yy, zz, theta, phi, size_hor, size_ver, light_domain = raw_angles_data_from_json(file_path)
  theta, phi = convert_to_angles(xx, yy, zz)
  xx, yy, zz = clean_3D_data_w_split(tt, xx, yy, zz, factor = 3, window = 5, threshold = -999)
  theta, phi = clean_2D_data_w_split(tt, theta, phi, factor = 3, window = 5, threshold = -999)
  complete_sample = np.stack([tt, xx, yy, zz, size_hor, size_ver]).T

  #segmenting
  skip = sample_duration * (1-overlap_factor)
  current_index = [0]
  end_index = np.nonzero(tt>=tt[0]+sample_duration)[0]

  sub_samples = []
  while end_index.size:
      # print(current_index[0], end_index[0])
      if end_index[0] - current_index[0] > samples_config['min_samples']: ### a threshold for a minimum number of datapoints in a sample
        sub_samples.append(complete_sample[current_index[0]:end_index[0], :])
      end_index = np.nonzero(tt>=tt[current_index[0]]+skip+sample_duration)[0]
      current_index = np.nonzero(tt>=tt[current_index[0]]+skip)[0]

  return sub_samples, light_domain

In [6]:
subfolders = os.listdir("track_data/")
subf_dict = {i:subfolders[i] for i in range(len(subfolders))}
labels_dict = {subfolders[i]:i for i in range(len(subfolders))}

### Extract samples

In [7]:
subf_dict

{0: 'airplane', 1: 'uav', 2: 'bird', 3: 'static-object'}

In [8]:
def extract_samples(folder, samples_config):
  subfolders = os.listdir(folder)
  subfolders_list = [subfolders[i] for i in samples_config['subfolders_ind']]

  samples_dict = {}
  samples_summary_dict = {}

  for subfolder in subfolders_list:
      print(subfolder)
      total_samples = 0
      subfolder_path = os.path.join(folder, subfolder)
      files = os.listdir(subfolder_path)
      for file in files:
          file_path = os.path.join(subfolder_path, file)
          sub_samples, light_domain = segment_file_by_time(file_path, samples_config)
          samples_dict[file] = sub_samples
          total_samples = total_samples + len(sub_samples)
      samples_summary_dict[subfolder] = total_samples
  print('samples summary:', samples_summary_dict)
  return samples_dict, samples_summary_dict

In [9]:
# samples_config = {
#     'sample_duration' : 15,
#     'overlap_factor' : 0.25,
#     'subfolders_ind' : [0, 1, 2, 3],
#     'min_samples' : 10
# }
# # sample_durations = [25]
# sample_durations = [3, 5, 10, 15, 20, 25]
# overlaps = [0, 0.25]
# for dur in sample_durations:
#   print('Sample Duration = ', dur)
#   samples_config['sample_duration'] = dur
#   for overlap in overlaps:
#     print('Overlap = ', overlap)
#     samples_config['overlap_factor'] = overlap

#     samples_dict, samples_summary_dict = extract_samples(folder, samples_config)
#     save_path = './Samples/samples_'+ str(dur) + 'ol' + str(overlap)
#     with open(save_path , 'wb') as f:
#         pickle.dump((samples_dict, samples_summary_dict), f)

## Helper transformations and feature functions (temp - to be moved to file)

In [10]:
def return_span(series):
  return series.max() - series.min()

In [11]:
def phi_theta_ratio(tt, data_theta, data_phi, pt_ratio_quants, dt=1, time_span = 3):
  """
  """
  new_tt = np.arange(tt[0], tt[-1]+dt, dt)

  interp_data_theta = np.interp(new_tt, tt, data_theta)
  pd_data_theta = pd.Series(interp_data_theta)
  rolling_data_theta = pd_data_theta.rolling(time_span)
  span_theta = rolling_data_theta.apply(return_span)

  interp_data_phi = np.interp(new_tt, tt, data_phi)
  pd_data_phi = pd.Series(interp_data_phi)
  rolling_data_phi = pd_data_phi.rolling(time_span)
  span_phi = rolling_data_phi.apply(return_span)
  span_ratios = np.arctan(span_phi/span_theta)
  span_ratios = span_ratios[~np.isnan(span_ratios)]
  span_ratios_quant = np.quantile(span_ratios, pt_ratio_quants)

  return span_ratios.min(), span_ratios.max(), span_ratios_quant

In [12]:
def PCA_angles_transformation(theta, phi):
  """returns the data, rotated so that the principle axis of the data in the xy plane is on x
  """
  tp_data = np.stack([theta, phi]).T

  pca = PCA(n_components=2, svd_solver='full')
  angles_projected = pca.fit_transform(tp_data)

  return angles_projected[:,0], angles_projected[:,1]
  # new_theta, new_phi = PCA_angles_transformation(cleaned_theta, cleaned_phi)

In [13]:
def angle_span_ratios(theta, phi):
  orig_ratio = (phi.max() - phi.min())/(theta.max() - theta.min())
  pca_theta, pca_phi = PCA_angles_transformation(theta, phi)
  pca_ratio = (pca_phi.max() - pca_phi.min())/(pca_theta.max() - pca_theta.min())
  return orig_ratio, pca_ratio

In [14]:
def smoothed(data, window = 5):
  """
  Args:
    data: A one dimensional array of data.
    window: The window of the convolution used for low-pass filtering.

  Returns:
    A smoothed version of the data
  """
  # Run a low pass filter.
  padded_data = np.pad(data, (window//2, window//2), 'edge')
  smooth_data = np.convolve(padded_data, np.ones(window) / window, mode='valid')
  return smooth_data

In [15]:
def derive(theta_data, phi_data, time):
  t_der = (time[:-1] + time[1:])/2
  der_theta = np.diff(theta_data)/np.diff(time)
  inf_ind1 = np.where(np.isinf(der_theta))
  nan_ind1 = np.where(np.isnan(der_theta))
  der_phi = np.diff(phi_data)/np.diff(time)
  inf_ind2 = np.where(np.isinf(der_phi))
  nan_ind2 = np.where(np.isnan(der_phi))
  inf_inds = np.union1d(inf_ind1, inf_ind2)
  nan_inds = np.union1d(nan_ind1, nan_ind2)
  inds = np.union1d(inf_inds, nan_inds)
  mask = np.ones(len(t_der), dtype=bool)
  mask[inds] = False

  return der_theta[mask], der_phi[mask], t_der[mask]

### Extract features

In [16]:
  t1 = np.arange(2, 5, 0.04)
  t2 = np.arange(5, 6, 0.04)
  part1 = (0.5/5**2)*t1**2 + 0.5
  part2 = 1 - 0.5*(t2-5)
  filt1 = np.flip(np.hstack([part1, part2]))
  t4 = np.arange(-5, -2, 0.04)
  t3 = np.arange(-6, -5, 0.04)
  part3 = -1 - 0.5*(t3+5)
  part4 = -(0.5/5**2)*t4**2 - 0.5
  filt2 = np.flip(np.hstack([part3, part4]))

In [17]:
def extract_features(sample):
  tt, xx, yy, zz, size_hor, size_ver = sample[:, 0], sample[:, 1], sample[:, 2], sample[:, 3], sample[:, 4], sample[:, 5]
  theta, phi = convert_to_angles(xx, yy, zz)
  # Basic transformations
  #standerdizing to start from 0
  phi = phi - phi[0]
  theta = theta - theta[0]
  # clean
  # xx, yy, zz = clean_3D_data_w_split(tt, xx, yy, zz, factor = 3, window = 5, threshold = -999)
  theta, phi = clean_2D_data_w_split(tt, theta, phi, factor = 3, window = 5, threshold = -999, replace_by = 'med')

  # interpolate
  delta = 0.04
  interp_tt, interp_theta = interpolate_data(tt, theta, dt=delta, fixed = False)
  interp_tt, interp_phi = interpolate_data(tt, phi, dt=delta, fixed = False)
  # interp_tt, interp_xx = interpolate_data(tt, xx, dt=delta, fixed = False)
  # interp_tt, interp_yy = interpolate_data(tt, yy, dt=delta, fixed = False)
  # interp_tt, interp_zz = interpolate_data(tt, zz, dt=delta, fixed = False)
  interp_tt, interp_size_hor = interpolate_data(tt, size_hor, dt=delta, fixed = False)
  interp_tt, interp_size_ver = interpolate_data(tt, size_ver, dt=delta, fixed = False)
  fix_interp_tt, fix_interp_theta = interpolate_data(tt, theta, dt=delta, fixed = True)
  fix_interp_tt, fix_interp_phi = interpolate_data(tt, phi, dt=delta, fixed = True)

  # derive
  interp_vel_theta, interp_vel_phi, interp_t_vel = derive(interp_theta, interp_phi, interp_tt)
  vel_theta, vel_phi, t_vel = derive(theta, phi, tt)
  abs_vel = np.sqrt(vel_theta**2 + vel_phi**2)
  acc_theta, acc_phi, t_acc = derive(vel_theta, vel_phi, t_vel)
  abs_acc = np.sqrt(acc_theta**2 + acc_phi**2)
  fix_vel_theta, fix_vel_phi, t_vel = derive(fix_interp_theta, fix_interp_phi, fix_interp_tt)

  # rolling window smoothing
  s_theta = smoothed(theta, window = 5)
  s_phi = smoothed(phi, window = 5)
  delta = 0.04
  # interp_tt, s_interp_theta = interpolate_data(tt, s_theta, dt=delta, fixed = False)
  # interp_tt, s_interp_phi = interpolate_data(tt, s_phi, dt=delta, fixed = False)
  # s_vel_theta = np.diff(s_interp_theta)/np.diff(interp_tt)
  # s_vel_phi = np.diff(s_interp_phi)/np.diff(interp_tt)
  # s_abs_vel = np.sqrt(s_vel_theta**2 + s_vel_phi**2)
  s_vel_theta, s_vel_phi, s_t_vel = derive(s_theta, s_phi, tt)
  s_abs_vel = np.sqrt(s_vel_theta**2 + s_vel_phi**2)
  s_vel_theta, s_vel_phi, s_t_vel = derive(s_theta, s_phi, tt)
  s_abs_vel = np.sqrt(s_vel_theta**2 + s_vel_phi**2)
  s_acc_theta, s_acc_phi, s_t_acc = derive(s_vel_theta, s_vel_phi, s_t_vel)
  s_abs_acc = np.sqrt(s_acc_theta**2 + s_acc_phi**2)

  # FEATURES
  returning_features = []
  feature_names = []
  # extreme values (scale dependent)
  max_size_hor = size_hor.max()
  max_size_ver = size_ver.max()
  max_elevation = phi.max()
  min_elevation = phi.min()
  max_theta_vel =  np.abs(vel_theta).max()
  max_phi_vel =  np.abs(vel_phi).max()
  max_vel = abs_vel.max()
  max_theta_acc =  np.abs(acc_theta).max()
  max_phi_acc =  np.abs(acc_phi).max()
  max_acc = abs_acc.max()
  returning_features.extend([max_elevation, min_elevation, max_theta_vel, max_phi_vel, max_vel, max_theta_acc, max_phi_acc, max_acc])
  feature_names.extend(['max_elevation', 'min_elevation', 'max_theta_vel', 'max_phi_vel', 'max_vel', 'max_theta_acc', 'max_phi_acc', 'max_acc'])

  s_max_elevation = s_phi.max()
  s_min_elevation = s_phi.min()
  s_max_theta_vel = np.abs(s_vel_theta).max()
  s_max_phi_vel = np.abs(s_vel_phi).max()
  s_max_vel = s_abs_vel.max()
  s_med_vel = np.median(s_abs_vel)
  s_max_theta_acc = np.abs(s_acc_theta).max()
  s_max_phi_acc = np.abs(s_acc_phi).max()
  s_max_acc = s_abs_acc.max()
  s_med_acc = np.median(s_abs_acc)
  returning_features.extend([s_max_elevation, s_min_elevation, s_max_theta_vel, s_max_phi_vel, s_max_vel, s_max_theta_acc, s_max_phi_acc, s_max_acc])
  feature_names.extend(['s_max_elevation', 's_min_elevation', 's_max_theta_vel', 's_max_phi_vel', 's_max_vel', 's_max_theta_acc', 's_max_phi_acc', 's_max_acc'])

  # statistics and noise
  theta_std = local_std(interp_theta, window = 10)
  phi_std = local_std(interp_phi, window = 10)
  theta_vel_std = local_std(interp_vel_theta, window = 10)
  phi_vel_std = local_std(interp_vel_phi, window = 10)
  returning_features.extend([theta_std, phi_std, theta_vel_std, phi_vel_std])
  feature_names.extend(['theta_std', 'phi_std', 'theta_vel_std', 'phi_vel_std'])

  # span
  theta_span = theta.max() - theta.min()
  phi_span = phi.max() - phi.min()
  returning_features.extend([theta_span, phi_span])
  feature_names.extend(['theta_span', 'phi_span'])

  # avg values
  med_elevation = np.median(phi)
  med_size_hor = np.median(size_hor)
  med_size_ver = np.median(size_ver)
  med_theta_vel =  np.median(vel_theta)
  med_phi_vel =  np.median(vel_phi)
  med_theta_acc =  np.median(acc_theta)
  med_phi_acc =  np.median(acc_phi)
  returning_features.extend([med_elevation, med_theta_vel, med_phi_vel, med_theta_acc, med_phi_acc, s_med_vel, s_med_acc])
  feature_names.extend(['med_elevation', 'med_theta_vel', 'med_phi_vel', 'med_theta_acc', 'med_phi_acc', 's_med_vel', 's_med_acc'])

  # Bounding box data
  returning_features.extend([max_size_hor, max_size_ver, med_size_hor, med_size_ver])
  feature_names.extend(['max_size_hor', 'max_size_ver', 'med_size_hor', 'med_size_ver'])


  # correlations
  # print(np.any(np.isinf(vel_theta)), np.any(np.isinf(vel_phi)))
  ascending_sig = np.max(np.convolve(fix_vel_phi/fix_vel_phi.max(), filt1, mode='valid'))
  decending_sig = np.max(np.convolve(fix_vel_phi/fix_vel_phi.min(), filt2, mode='valid'))

  pt_vel_corr, _ = pearsonr(np.abs(vel_theta), np.abs(vel_phi))
  pt_acc_corr, _ = pearsonr(np.abs(acc_theta), np.abs(acc_phi))

  returning_features.extend([ascending_sig, decending_sig, pt_vel_corr, pt_acc_corr])
  feature_names.extend(['ascending_sig', 'decending_sig', 'pt_vel_corr', 'pt_acc_corr'])

  # FFT

  # yf = fft(fix_vel_phi)
  # yf_trimmed_abs = np.abs(yf[0:len(yf)//2+1])

  # ratios
  orig_ratio, pca_ratio = angle_span_ratios(theta, phi)
  std_vs_span = np.sqrt((theta_std**2 + phi_std**2)/(theta_span**2 + phi_span**2))
  returning_features.extend([orig_ratio, pca_ratio, std_vs_span])
  feature_names.extend(['orig_ratio', 'pca_ratio', 'std_vs_span'])

  # scale independent features
  # measuring geometric curvature

  discrete_angle_dist = np.sqrt(np.diff(interp_theta)**2 + np.diff(interp_phi)**2)
  cumsum_angles_dist = np.cumsum(discrete_angle_dist)
  path_total_length = cumsum_angles_dist[-1]
  cum_dist = np.hstack([0, cumsum_angles_dist])

  angles = np.stack([interp_theta, interp_phi]).T
  eucl_dist = cdist(angles, angles, 'euclidean') + np.ones([len(interp_theta), len(interp_theta)])*1e-6 #added to avoid devision by 0

  path_dist = np.abs(cum_dist[:, None] - cum_dist)
  dist_ratio = path_dist/eucl_dist
  distance_buffer = 5 #removing ratios of points close to one another, since this may be noisy
  all_curve_ratios = sum((dist_ratio[i,i+distance_buffer:].tolist() for i in range(dist_ratio.shape[0])), [])
  curve_quants = np.arange(0.1, 0.95, 0.1)
  curve_quantiles = np.quantile(all_curve_ratios, curve_quants)
  max_curve_ratio = np.max(all_curve_ratios)
  returning_features.extend([*curve_quantiles, max_curve_ratio])
  q_names = ['curve quant ' + str(round(q, 2)) for q in curve_quants]
  feature_names.extend(q_names + ['max_curve_ratio'])

  # measuring angles ratios (quantiles):
  # e.g. object rising fast without changing azimuth, or changing azimuth without elevation
  pt_ratio_quants = np.arange(0.1, 0.95, 0.1)
  min_span_ratios, max_span_ratios, span_ratios_quant = phi_theta_ratio(tt, theta, phi, pt_ratio_quants, dt=1, time_span = 3)
  returning_features.extend([*span_ratios_quant, max_span_ratios, min_span_ratios])
  q_names = ['pt ratio quant ' + str(round(q, 2)) for q in pt_ratio_quants]
  feature_names.extend(q_names + ['max_span_ratios', 'min_span_ratios'])

  # velocity and acceleration profiles (ratios)
  motion_quants = np.arange(0.1, 0.95, 0.1)
  vel_quantiles = np.quantile(s_abs_vel, motion_quants)
  vel_profile = vel_quantiles/s_med_vel
  acc_quantiles = np.quantile(s_abs_acc, motion_quants)
  acc_profile = acc_quantiles/s_med_acc
  q_vel_names = ['vel quant ' + str(round(q, 2)) for q in motion_quants]
  q_acc_names = ['acc quant ' + str(round(q, 2)) for q in motion_quants]
  returning_features.extend([*vel_profile, *acc_profile])
  feature_names.extend(q_vel_names + q_acc_names)

  return returning_features, feature_names

### Prepare dataset

In [18]:
def prepare_dataset(split_config):
  print('collecting samples')
  # subfolders_list = [subf_dict[i] for i in split_config['subfolders_ind']]
  subfolders_list = [subf_dict[i] for i in np.arange(4)]
  save_path = './Samples/samples_'+ str(split_config['sample_duration']) + 'ol' + str(split_config['overlap_factor'])
  with open(save_path , 'rb') as f:
    samples_dict, samples_summary_dict = pickle.load(f)

  # files_labels = []
  all_samples = []
  samples_labels = []
  samples_filenames = []
  samples_light = []
  database_summary_dict = {}

  for subfolder in subfolders_list:
    subfolder_path = os.path.join(folder, subfolder)
    files = os.listdir(subfolder_path) # Can also be taken from samples_dict keys
    # files_labels.extend([labels_dict[subfolder]]*len(files))
    n_subfolder_samples = 0
    for file in files:
      light_domain = file[:3]
      sub_samples = samples_dict[file]
      all_samples.extend(sub_samples)
      sub_labels = [labels_dict[subfolder]]*len(sub_samples)
      samples_labels.extend(sub_labels)
      sub_light = [light_domain]*len(sub_samples)
      samples_light.extend(sub_light)
      sub_file = [file]*len(sub_samples)
      samples_filenames.extend(sub_file)
      n_subfolder_samples = n_subfolder_samples + len(sub_samples)
    database_summary_dict[subfolder] = n_subfolder_samples
  print('extracting features')
  data = []
  for i, sample in enumerate(all_samples):
    data_row, features = extract_features(sample)
    data.append(data_row)

  df = pd.DataFrame(data, columns=features)
  df['light'] = samples_light
  df['file'] = samples_filenames
  df['label'] = samples_labels

  print('Done')
  return df, features

### Split

In [19]:
def split_by_files(df, df0, split_config):
  files = np.unique(df['file'].values)
  files_ind = [np.where(df['file'].values == f)[0][0] for f in files]
  files_labels = df['label'].values[files_ind]
  rs = split_config['random_state']
  ts = split_config['test_split_files']
  files_train, files_test = train_test_split(files, test_size=ts, random_state=rs, stratify=files_labels)
  train_df = df[df['file'].isin(files_train)]
  test_df = df0[df0['file'].isin(files_test)]
  return train_df, test_df

In [20]:
print(labels_dict)
print(subf_dict)

{'airplane': 0, 'uav': 1, 'bird': 2, 'static-object': 3}
{0: 'airplane', 1: 'uav', 2: 'bird', 3: 'static-object'}


### Evaluation

In [21]:
def evaluate(labels, predictions, caption, plot_cm , print_scores):

  classes = np.union1d(labels, predictions)
  tick_names = [subf_dict[c] for c in classes]
  # Confusion Matrix - Multi class
  cm = confusion_matrix(labels, predictions)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=tick_names)
  if plot_cm:
    disp.plot()
    disp.ax_.set_title(caption)
    plt.show()

  # Scores #
  # f1_s = f1_score(labels, predictions, sample_weight=None, average = 'weighted', zero_division='warn')
  report = classification_report(labels, predictions, target_names = tick_names, output_dict=True, digits=3, zero_division = 0)

  f1_all = report['weighted avg']['f1-score']

  if print_scores == True:
      print(classification_report(labels, predictions, target_names = tick_names,digits=3, zero_division = 0))
      # print(f1_s)
      print('Average F1 score for all classes = ', f1_all)
      print('------------------------')
      print('UAV report')
      print('------------------------')
      print('precision = ', report['uav']['precision'])
      print('recall = ', report['uav']['recall'])
      print('F1 = ', report['uav']['f1-score'])
      print('support = ', report['uav']['support'])

  return report

### Model

In [22]:
class MLP(torch.nn.Module):
    def __init__(self, config):
        super(MLP, self).__init__()

        self.do = config['dropout']
        self.num_classes = config['num_classes']
        self.num_features = config['num_features']

        self.input_layer = torch.nn.Linear(in_features=self.num_features, out_features = config['layers'][0])
        self.fc_layers = torch.nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(config['layers'][:-1], config['layers'][1:])):
            self.fc_layers.append(torch.nn.Linear(in_size, out_size))

        self.dropout = torch.nn.Dropout(self.do)
        self.logits = torch.nn.Linear(in_features=config['layers'][-1] , out_features=self.num_classes)

    def forward(self, samples):
        samples = self.dropout(samples)
        mlp_vector = self.input_layer(samples)
        mlp_vector = torch.nn.ReLU()(mlp_vector)
        # torch.cat([user_embedding_mlp, item_embedding_mlp], dim=-1)  # the concat latent vector

        for idx, _ in enumerate(range(len(self.fc_layers))):
            mlp_vector = self.dropout(mlp_vector)
            mlp_vector = self.fc_layers[idx](mlp_vector)
            mlp_vector = torch.nn.ReLU()(mlp_vector)

        mlp_vector = self.dropout(mlp_vector)
        logits = self.logits(mlp_vector)
        # output = self.sigmoid(logits).squeeze()
        return logits

In [23]:
def initialize_weights(model):
  if isinstance(model, nn.Linear):
    nn.init.xavier_uniform_(model.weight, gain=1)
    nn.init.constant_(model.bias, 0)

### Training class

In [24]:
class Training(object):

  def __init__(self, model, config):
    self.config = config
    self.model = model.to(self.config['device'])
    self.optimizer = config['optimizer_type'](model.parameters(), **config['optimizer_parameter'])
    self.criterion = config['criterion'](weight = config['class_weights']).to(self.config['device'])
    self.dl_train = dl_train

  def train(self):
    self.train_loss_history = []
    self.eval_loss_history = []
    self.eval_uav_precision_history = []
    self.eval_uav_recall_history = []
    self.eval_uav_f1_history = []
    self.eval_total_f1_history = []

    epochs_without_improvement = 0
    best_f1 = None

    train_start = time()

    for epoch in range(self.config['n_epochs']):
    # for epoch in range(1):
      self.train_epoch()
      self.evaluate_epoch(dl_val)
      if self.config['verbose']:
        print(f'epoch {epoch}: loss = {self.train_loss_history[-1]}, f1 = {self.eval_uav_f1_history[-1]}')
      #check for early stopping
      if not best_f1 or self.eval_uav_f1_history[-1] > best_f1:
        best_f1 = self.eval_uav_f1_history[-1]
        # print("best uav f1 ", best_f1)
        best_precision = self.eval_uav_precision_history[-1]
        best_recall = self.eval_uav_recall_history[-1]
        best_loss = self.eval_loss_history[-1]
        best_total_f1 = self.eval_total_f1_history[-1]
        epochs_without_improvement = 0
        #print ("Achieved lower validation loss, save model at epoch number {} ".format(epoch + 1) )
        best_model = copy.deepcopy(self.model.state_dict())
      else:
        epochs_without_improvement += 1

      if epochs_without_improvement == self.config['early_stopping']:
        if self.config['verbose']:
            print('\nEarly stoping after {} epochs. validation f1 did not imporve for more than {} epcochs'.format(epoch, self.config['early_stopping']))
        break
    self.training_time = time() - train_start

    # load best model and best performance
    self.model.load_state_dict(best_model)
    if self.config['verbose']:
        print('\nFinished Training:')
        print('Best metrics are:')
        # print('Evaluation LogLoss = '.format(best_loss))

        print(f'Best f1 eval = {best_f1}')
        print(f'Best recall eval= {best_recall}')
        print(f'Best precision eval= {best_precision}')
        print(f'Best total f1 eval= {best_total_f1}')

  def train_epoch(self):
    self.epoch_train_loss   = 0
    self.model.train() # train mode
    # ii = 0
    for batch in tqdm(self.dl_train, disable=(not self.config['verbose'])):
      # ii +=1
      self.train_batch(batch)
      # if ii>1:
      #   break
    self.train_loss_history.append(self.epoch_train_loss/len(self.dl_train))

  def train_batch(self, batch):
    samples, labels = batch
    # Send tensors to GPU
    samples = samples.to(device)
    labels = labels.to(device)

    # pred = self.model(user_indices, item_indices).squeeze().type(torch.DoubleTensor)
    pred = self.model(samples)#.type(torch.float64)#.type(torch.DoubleTensor)
    # print(pred.get_device())
    # print('pred = ', pred)

    # loss = self.criterion(pred, labels.float()) # can make label float in creation!!!!!!!!!!!!!!!!!!!!
    # labels = labels[:, None]
    # print('labels = ', labels)
    loss = self.criterion(pred, labels)
    # loss = self.criterion(pred, labels)

    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.epoch_train_loss += loss.item()

  def evaluate_epoch(self, dl_eval):
    self.eval_labels = []
    self.eval_predictions = []
    self.epoch_eval_loss = 0

    self.model.eval() #evaluation mode
    with torch.no_grad():
      for batch in tqdm(dl_eval, disable=(not self.config['verbose'])):
      # for batch in tqdm(dl_val, disable=(not self.config['verbose'])):
      # for batch in tqdm(dl_test, disable=(not self.config['verbose'])):
        self.eval_batch(batch)
    # print(self.eval_labels)
    # print(self.eval_predictions)
    # print(type(self.eval_labels))
    # print(type(self.eval_predictions))
    # print(len(self.eval_labels))
    # print(len(self.eval_predictions))

    # print('self.eval_labels ', self.eval_labels)
    # print('self.eval_predictions ', self.eval_predictions)
    classes = np.union1d(self.eval_labels, self.eval_predictions)
    tick_names = [subf_dict[c] for c in classes]
    report = classification_report(self.eval_labels, self.eval_predictions, target_names = tick_names, output_dict=True, digits=3, zero_division = 0)
    # print(report)
    self.report = report
    self.eval_loss_history.append(self.epoch_eval_loss/len(dl_val))
    self.eval_uav_precision_history.append(report['uav']['precision'])
    self.eval_uav_recall_history.append(report['uav']['recall'])
    self.eval_uav_f1_history.append(report['uav']['f1-score'])
    self.eval_total_f1_history.append(report['weighted avg']['f1-score'])

  def eval_batch(self, batch):
    samples, labels = batch
    # Send tensors to GPU
    samples = samples.to(self.config['device'])
    labels = labels.to(self.config['device'])

    pred = self.model(samples).detach()
    eval_loss = self.criterion(pred, labels)
    # pred = self.model(samples).detach().cpu()
    # print(labels)
    # print(torch.argmax(pred, dim = 1))
    pred = pred.cpu()
    labels = labels.cpu()
    self.eval_predictions.extend(torch.argmax(pred, dim = 1))
    self.eval_labels.extend(labels)

    # eval_loss = self.criterion(pred.type(torch.float64), labels)
    self.epoch_eval_loss += eval_loss.item()

In [25]:
def plot_training_summary(training_model):
  epochs = np.arange(len(training_model.train_loss_history)) + 1
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5))
  ax1.plot(epochs, training_model.train_loss_history, 'r', label = 'train loss')
  ax1.plot(epochs, training_model.eval_loss_history, 'orange', label = 'eval loss')
  ax1.set_xlabel('Epochs')
  ax1.set_ylabel('Loss')
  ax1.legend()
  ax2.plot(epochs, training_model.eval_uav_precision_history, 'r', label = 'uav precision')
  ax2.plot(epochs, training_model.eval_uav_recall_history, 'g', label = 'uav recall')
  ax2.plot(epochs, training_model.eval_uav_f1_history, 'b', label = 'uav f1')
  ax2.plot(epochs, training_model.eval_total_f1_history, 'm', label = 'total f1')
  ax2.set_xlabel('Epochs')
  ax2.set_ylabel('Metrics')
  ax2.legend()

## Experiments

In [26]:
# record_columns = ['datetime', 'Sample Duration', 'Test Files', 'Split Config', 'Features Config', 'Model', 'Model Config', 'Evaluation Report', 'UAV precision', 'UAV recall', 'UAV f1', 'Total f1']
# exp_record = pd.DataFrame(columns = record_columns)

In [27]:
def conf2str(split_config, features_config):
  conf_str = 'sd' + str(split_config['sample_duration']) \
  +'of' + str(split_config['overlap_factor']) \
  +'ms' + str(split_config['min_samples']) \
  +'f_date' + str(features_config['date'])
  return conf_str

In [28]:
features_dict = {
    'extremum' : np.arange(0,16),
    'std' : np.arange(16,20),
    'span' : np.arange(20,22),
    'med' : np.arange(22,29),
    'bbox' : np.arange(29,33),
    'corr_sig' : np.arange(33,37),
    'full_ratio' : np.arange(37,40),
    'curve' : np.arange(40,50),
    'pt_ratio' : np.arange(50,61),
    'vel_acc_profile' : np.arange(61,79)
}

In [29]:
def get_datasets(split_config, features_config):
  conf_str = conf2str(split_config, features_config)
  database_path = './Features Databases/database_'+ conf_str
  if os.path.isfile(database_path):
    with open(database_path , 'rb') as f:
      df, features = pickle.load(f)
      # print('Loading dataset')
  else:
    df, features = prepare_dataset(split_config)
    with open(database_path , 'wb') as f:
      pickle.dump((df, features), f)

  #retrieving 0 overlap matching df
  ol0_config = split_config.copy()
  ol0_config['overlap_factor'] = 0
  conf_str = conf2str(ol0_config, features_config)

  database0_path = './Features Databases/database_'+ conf_str
  if os.path.isfile(database0_path):
    with open(database0_path , 'rb') as f:
      df0, features = pickle.load(f)
      # print('Loading dataset')
  else:
    df0, features = prepare_dataset(ol0_config)
    with open(database0_path , 'wb') as f:
      pickle.dump((df0, features), f)
  return df, df0, features

In [30]:
def scale_dataset(df, df0, features):
  # scaling by df
  for feature in features:
    if df[feature].std() > 0:
      col_min = df[feature].min()
      col_max = df[feature].max()
      df[feature] = (df[feature] - col_min) / (col_max - col_min)
      df0[feature] = (df0[feature] - col_min) / (col_max - col_min)
      # df[feature] = (df[feature]-df[feature].mean())/df[feature].std()
      # df0[feature] = (df0[feature]-df0[feature].mean())/df0[feature].std()
  return df, df0

In [31]:
record_columns = ['datetime', 'Sample Duration', 'Test Files', 'Split Config', 'Features Config', 'Model', 'Model Config', 'Training Config', 'Evaluation Report', 'UAV precision', 'UAV recall', 'UAV f1', 'Total f1']
exp_record = pd.DataFrame(columns = record_columns)

torch.manual_seed(42)
np.random.seed(42)

split_config = {
    'sample_duration' : 15,
    'overlap_factor' : 0.25,
    'subfolders_ind' : [0, 1, 2, 3],
    'min_samples' : 10,
    'random_state' : 24,
    'test_split_files' : 0.2, # 2 for n files
    'kfold' : 5
  }

features_config = {
    'date' : 20_1_2024,
    'extremum' : True,
    'std' : True,
    'span' : True,
    'med' : True,
    'bbox' : True,
    'corr_sig' : True,
    'full_ratio' : True,
    'curve' : True,
    'pt_ratio' : True,
    'vel_acc_profile' : True
  }


# mlp_config = {'num_classes' : len(split_config['subfolders_ind']),'layers': [128,64,32,16], 'dropout': 0, 'model_random_state' : 42}
mlp_config = {'num_classes' : len(split_config['subfolders_ind']),'layers': [256, 128, 64, 32, 16], 'dropout': 0, 'model_random_state' : 42}
training_config = {'batch_size': 16, 'optimizer_type': Adam, 'optimizer_parameter': {'lr': 0.0001}, 'class_weights' : torch.tensor([0.1, 0.5, 0.25, 0.15]), \
                          'criterion' : nn.CrossEntropyLoss, 'n_epochs' : 250, 'early_stopping' : 80, 'verbose' : False, 'device' : device}

# split_random_states = [32]
# sample_durations = [20]
model_random_states = [42]
split_random_states = [42]
sample_durations = [5, 10, 15, 20, 25]
# sample_durations = [10]
# layerss = [[256, 128, 64, 32, 16]]
layerss = [[256, 256, 128, 32]]
# layerss = [[80, 80]]
# dropouts = [0, 0.25, 0.5]
dropouts = [0.25]
lrs = [0.00005, 0.0001, 0.0003, 0.001, 0.005]
batch_sizes = [16, 32, 64]
# lrs = [0.005]
# batch_sizes = [32]
class_weightss = [torch.tensor(float('nan')), torch.tensor([0.15, 0.5, 0.25, 0.1])]
# class_weightss = [torch.tensor([0.15, 0.5, 0.25, 0.1])]

model_name = 'MLP'
model_config = mlp_config
now = datetime.now()
date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
for mrs in model_random_states:
  model_config['model_random_state'] = mrs
  for rs in split_random_states:
    split_config['random_state'] = rs
    for dur in sample_durations:
      split_config['sample_duration'] = dur
      for layers in layerss:
        model_config['layers'] = layers
        for do in dropouts:
          model_config['dropout'] = do
          for lr in lrs:
            training_config['optimizer_parameter']['lr'] = lr
            for batch_size in batch_sizes:
              training_config['batch_size'] = batch_size
              for cw in class_weightss:
                training_config['class_weights'] = cw
                # print(split_config)
                # print(model_config)
                #Don't change split_config beyond this point
                df, df0, features = get_datasets(split_config, features_config)
                # df.fillna(0, inplace=True)
                # df0.fillna(0, inplace=True)
                feature_columns = np.empty(0)
                for key in features_config.keys():
                  if features_config[key] == True:
                    feature_columns = np.hstack([feature_columns, features_dict[key]])

                df, df0 = scale_dataset(df, df0, features)
                train_df, test_df = split_by_files(df, df0, split_config)
                test_df = test_df[test_df['label'].isin(split_config['subfolders_ind'])] # removing the folders we don't want to analyze (in split_config)
                test = torch.tensor(test_df.iloc[:, feature_columns].values, dtype = torch.float32)
                test_labels = torch.tensor(test_df['label'].values)
                test_data = list(zip(test, test_labels))
                dl_test = DataLoader(test_data, batch_size = training_config['batch_size'])


                train_df, val_df = split_by_files(train_df, df0, split_config)
                train_df = train_df[train_df['label'].isin(split_config['subfolders_ind'])] # removing the folders we don't want to analyze (in split_config)
                train_df = train_df.sample(frac = 1, random_state = model_config['model_random_state']) #shuffled df
                train = torch.tensor(train_df.iloc[:, feature_columns].values, dtype = torch.float32)
                val_df = val_df[val_df['label'].isin(split_config['subfolders_ind'])] # removing the folders we don't want to analyze (in split_config)
                val = torch.tensor(val_df.iloc[:, feature_columns].values, dtype = torch.float32)
                train_labels = torch.tensor(train_df['label'].values)
                val_labels = torch.tensor(val_df['label'].values)

                train_data = list(zip(train, train_labels))
                dl_train = DataLoader(train_data, batch_size = training_config['batch_size'])
                val_data = list(zip(val, val_labels))
                dl_val = DataLoader(val_data, batch_size = training_config['batch_size'])

                torch.manual_seed(42)
                np.random.seed(42)
                mlp_config['num_features'] = len(feature_columns)
                model_MLP = MLP(mlp_config)
                model_MLP.apply(initialize_weights)
                # calculate class weights for training
                if torch.any(torch.isnan(training_config['class_weights'])):
                  u_labels, counts = torch.unique(train_labels, return_counts=True)
                  class_weights = 1/counts
                  training_config['class_weights'] = class_weights

                # Training
                training_mlp = Training(model_MLP, training_config)
                training_mlp.train() #end of training loads the best model
                # plot_training_summary(training_mlp)
                print('finished training')

                training_mlp.evaluate_epoch(dl_val)
                # aggregate metrics
                uav_precision_m = training_mlp.report['uav']['precision']
                uav_recall_m = training_mlp.report['uav']['recall']
                uav_f1_m = training_mlp.report['uav']['f1-score']
                weighted_f1_m = training_mlp.report['weighted avg']['f1-score']
                # finally with test data
                training_mlp.evaluate_epoch(dl_test)
                test_report = training_mlp.report

                # wrong_ind = np.where(predictions != test_labels)
                # print('wrong on:')
                # print(test_df['file'].iloc[wrong_ind])
                test_files = np.unique(test_df['file'])
                exp_record.loc[len(exp_record)] = [date_time, dur, test_files, split_config.copy(), features_config, model_name, model_config.copy(), copy.deepcopy(training_config),
                                  test_report, uav_precision_m, uav_recall_m, uav_f1_m, weighted_f1_m]
                # exp_record.to_csv(exp_record_path, index = False)

finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished training
finished t

# New Section

In [32]:
# training_mlp.report

In [33]:
date_time_ = str(date_time).replace("/","_")
results_path = './Features Extraction Results/features_extraction_results_' + date_time_

with open(results_path , 'wb') as f:
  pickle.dump(exp_record, f)

In [34]:
# for rep in exp_record['Evaluation Report']: #mean std normalization
#   print(rep)

In [35]:
# for rep in exp_record['Evaluation Report']: #min_max normalization
#   print(rep)

In [36]:
for rep in exp_record['Evaluation Report']: #min_max normalization
  print(rep)

{'airplane': {'precision': 0.9107142857142857, 'recall': 0.884393063583815, 'f1-score': 0.8973607038123167, 'support': 173}, 'uav': {'precision': 0.8470588235294118, 'recall': 0.6233766233766234, 'f1-score': 0.7182044887780549, 'support': 231}, 'bird': {'precision': 0.5238095238095238, 'recall': 0.8461538461538461, 'f1-score': 0.6470588235294118, 'support': 39}, 'static-object': {'precision': 0.9191073919107392, 'recall': 0.9762962962962963, 'f1-score': 0.94683908045977, 'support': 675}, 'accuracy': 0.8846153846153846, 'macro avg': {'precision': 0.8001725062409901, 'recall': 0.8325549573526453, 'f1-score': 0.8023657741448884, 'support': 1118}, 'weighted avg': {'precision': 0.8891325765940841, 'recall': 0.8846153846153846, 'f1-score': 0.8814850734304591, 'support': 1118}}
{'airplane': {'precision': 0.8857142857142857, 'recall': 0.8959537572254336, 'f1-score': 0.8908045977011495, 'support': 173}, 'uav': {'precision': 0.7195121951219512, 'recall': 0.7662337662337663, 'f1-score': 0.7421383